In [59]:
import pandas as pd
from difflib import get_close_matches
import warnings
from math import sin, cos, sqrt, atan2, radians
from tqdm.notebook import tqdm

In [2]:
# data comes from https://simplemaps.com/data/us-cities
# all uses of this tool must provide a reference to this source
city_df = pd.read_csv('uscities.csv')

In [3]:
# use get closes matches to search for city names
get_close_matches('podunk', city_df['city'], n=10)

['Funk']

In [4]:
# small function to return states with that
def get_closests_cities(a_city, n=10):
    '''
    Return the n closest matches to a given city name as a dataframe
    '''
    return city_df.loc[city_df['city'].isin(get_close_matches(a_city, city_df['city'], n=n))]

In [5]:
get_closests_cities('Funk')

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
1800,Frank,Frank,WV,West Virginia,54075,Pocahontas,54075,Pocahontas,38.5462,-79.8080,65.0,65.0,polygon,False,False,America/New_York,3,26264 24920,1840025652
8197,Funk,Funk,NE,Nebraska,31137,Phelps,31137,Phelps,40.4630,-99.2508,185.0,271.0,polygon,False,True,America/Chicago,3,68940,1840012078
9958,Hunker,Hunker,PA,Pennsylvania,42129,Westmoreland,42129,Westmoreland,40.2041,-79.6155,280.0,427.0,polygon,False,True,America/New_York,3,15639,1840001138
15417,Fouke,Fouke,AR,Arkansas,5091,Miller,05091,Miller,33.2622,-93.8852,867.0,246.0,polygon,False,True,America/Chicago,3,71837,1840013752
20169,Funkstown,Funkstown,MD,Maryland,24043,Washington,24043,Washington,39.6089,-77.7082,877.0,921.0,polygon,False,True,America/New_York,3,21734 21740,1840005609
22228,Kunkle,Kunkle,OH,Ohio,39171,Williams,39171,Williams,41.6359,-84.4942,287.0,395.0,polygon,False,False,America/New_York,3,43531 43501,1840026276
24398,Funkley,Funkley,MN,Minnesota,27007,Beltrami,27007,Beltrami,47.7860,-94.4266,5.0,2.0,polygon,False,True,America/Chicago,3,56630,1840007698
25842,Susank,Susank,KS,Kansas,20009,Barton,20009,Barton,38.6407,-98.7745,32.0,124.0,polygon,False,True,America/Chicago,3,67544,1840009651
26048,Falun,Falun,KS,Kansas,20169,Saline,20169,Saline,38.6747,-97.7510,80.0,27.0,polygon,False,False,America/Chicago,3,67442,1840024598
27217,Furman,Furman,SC,South Carolina,45049,Hampton,45049,Hampton,32.6812,-81.1877,219.0,27.0,polygon,False,True,America/New_York,3,29921 29918,1840016925


In [38]:
def get_single_city(a_city, a_state):
    '''
    For a given city state pair, return the closest match
    First looks for exact match in state, 
    then returns closest match for city
    State should be in two letter state ID format
    '''
    city_subset = city_df.loc[city_df['state_id']==a_state.upper()]
    # get closest match in state
    candidate_city = get_close_matches(a_city, city_subset['city'], n=1)[0]
    candidate_city = city_subset[city_subset['city']==candidate_city].reset_index()
    return candidate_city.iloc[0]

In [43]:
get_single_city('Frenk', 'WV')

index                          1800
city                          Frank
city_ascii                    Frank
state_id                         WV
state_name            West Virginia
county_fips                   54075
county_name              Pocahontas
county_fips_all               54075
county_name_all          Pocahontas
lat                         38.5462
lng                         -79.808
population                       65
density                          65
source                      polygon
military                      False
incorporated                  False
timezone           America/New_York
ranking                           3
zips                    26264 24920
id                       1840025652
Name: 0, dtype: object

In [44]:
def get_city_coords(a_city, a_state):
    '''
    Get lat and lng for a city with closest name match
    '''
    a_city = get_single_city(a_city, a_state)
    city_name = a_city['city']
    lat = a_city['lat']
    lng = a_city['lng']
    return city_name, a_state.upper(), lat, lng

In [45]:
get_city_coords('Funkley', 'MN')

('Funkley', 'MN', 47.786, -94.4266)

In [46]:
import math

def distance(origin, destination):
    '''
    Return distance between two sets of coords
    in KM
    '''
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [53]:
def city_distance(city_1, state_1, city_2, state_2):
    '''
    Return distance between two cities in KM
    '''
    city_1, state_1, lat1, lng1 = get_city_coords(city_1, state_1)
    city_2, state_2, lat2, lng2 = get_city_coords(city_2, state_2)
    distance_dict = {'city_1': city_1, 
                     'state_1': state_1, 
                     'city_2': city_2, 
                     'state_2': state_2}
    distance_dict['distance'] = distance((lat1, lng1), (lat2, lng2))
    return distance_dict

In [54]:
city_distance('Funkley', 'MN', 'Kunkle', 'OH')

{'city_1': 'Funkley',
 'state_1': 'MN',
 'city_2': 'Kunkle',
 'state_2': 'OH',
 'distance': 1039.5226490891246}

In [80]:
def all_pairs_distances(city_list, state_list):
    assert len(city_list)==len(state_list), "City and state lists must be the same length"
    distances_list = []
    length = len(city_list)
    for city_1, state_1 in tqdm(zip(city_list, state_list), total=length):
        for city_2, state_2 in zip(city_list, state_list):
            a_distance = city_distance(city_1, state_1, city_2, state_2)
            if int(a_distance['distance'])!=0:
                distances_list.append(a_distance)
    return pd.DataFrame(distances_list)

In [81]:
some_cities = ['Palo Alto', 'Seattle', 'Boston', 'New York', 
               'San Francisco', 'Valley View', 'Rising City',
               'Purcell', 'Concordia', 'Little Rock', 'Miami',
               'Ann Arbor', 'Duluth', 'Denton', 'New Orleans',
               'Portland', 'Portland', 'Memphis', 'Hurricane',
               'Aspen', 'Truth or Consequences']
some_state = ['CA', 'WA', 'MA', 'NY', 'CA', 'TX', 'NE',
              'OK', 'KS', 'AR', 'FL', 'MI', 'MN', 'TX', 
              'LA', 'OR', 'ME', 'TN', 'UT', 'CO', 'NM']

In [82]:
distances = all_pairs_distances(some_cities, some_state)

In [83]:
distances

,city_1,state_1,city_2,state_2,distance
0,Palo Alto,CA,Seattle,WA,1137.594252
1,Palo Alto,CA,Boston,MA,4327.469306
2,Palo Alto,CA,New York,NY,4128.446628
3,Palo Alto,CA,San Francisco,CA,48.251480
4,Palo Alto,CA,Valley View,TX,2298.325331
...,...,...,...,...,...
415,Truth or Consequences,NM,Portland,OR,1900.554246
416,Truth or Consequences,NM,Portland,ME,3393.281750
417,Truth or Consequences,NM,Memphis,TN,1602.536658
418,Truth or Consequences,NM,Hurricane,UT,707.419132
